In [123]:
import os
import glob
import ddsp.training
from matplotlib import pyplot as plt
import IPython.display as ipd
import gin
import numpy as np
import utils
import tensorflow as tf

%matplotlib inline
sample_rate = 16000

## Set paths

### This DRIVE_DIR must includes your audio files

In [124]:
DRIVE_DIR = './Pretrained_ Models_for_T2/training'  
assert os.path.exists(DRIVE_DIR)

In [125]:
# create all directories leading up to the given directory that do not exist already. 
# If the given directory already exists, ignore the error.
DATA_DIR = os.path.join(DRIVE_DIR, 'data')
!mkdir -p "$DATA_DIR"
AUDIO_DIR = os.path.join(DATA_DIR, 'audio')
AUDIO_FILEPATTERN = AUDIO_DIR + '/*'
!mkdir -p "$AUDIO_DIR"
# folder to save the model
SAVE_DIR = os.path.join(DRIVE_DIR, 'ddsp-solo-instrument')
!mkdir -p "$SAVE_DIR"

## Prepare Dataset


In [126]:
mp3_files = glob.glob(os.path.join(DRIVE_DIR, '*.mp3'))
wav_files = glob.glob(os.path.join(DRIVE_DIR, '*.wav'))
audio_files = mp3_files + wav_files

for fname in audio_files:
  target_name = os.path.join(AUDIO_DIR, 
                             os.path.basename(fname).replace(' ', '_'))
  print('Copying {} to {}'.format(fname, target_name))
  !cp "$fname" "$target_name"

Copying ./Pretrained_ Models_for_T2/training/sax_1.wav to ./Pretrained_ Models_for_T2/training/data/audio/sax_1.wav


In [ ]:
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
!mkdir -p "$TRAIN_DIR"
TRAIN_TFRECORD = TRAIN_DIR + '/train.tfrecord'

!ddsp_prepare_tfrecord \
--input_audio_filepatterns="$AUDIO_FILEPATTERN "\
--output_tfrecord_path="$TRAIN_TFRECORD" \
--num_shards=10 \
--alsologtostderr

I0321 13:24:40.506776 4575448576 translations.py:641] ==================== <function annotate_downstream_side_inputs at 0x7fd0b40e6dc0> ====================
I0321 13:24:40.507302 4575448576 translations.py:641] ==================== <function fix_side_input_pcoll_coders at 0x7fd0b40e6ee0> ====================
I0321 13:24:40.507780 4575448576 translations.py:641] ==================== <function lift_combiners at 0x7fd0b40e81f0> ====================
I0321 13:24:40.508028 4575448576 translations.py:641] ==================== <function expand_sdf at 0x7fd0b40e83a0> ====================
I0321 13:24:40.508307 4575448576 translations.py:641] ==================== <function expand_gbk at 0x7fd0b40e8430> ====================
I0321 13:24:40.508681 4575448576 translations.py:641] ==================== <function sink_flattens at 0x7fd0b40e8550> ====================
I0321 13:24:40.508837 4575448576 translations.py:641] ==================== <function greedily_fuse at 0x7fd0b40e85e0> ====================


In [ ]:
TRAIN_TFRECORD_FILEPATTERN = TRAIN_DIR + '/train.tfrecord*'
print(TRAIN_TFRECORD_FILEPATTERN )
tfre_files = glob.glob(TRAIN_DIR + '/train.tfrecord*')
print(tfre_files)

In [ ]:
data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)
PICKLE_FILE_PATH = os.path.join(SAVE_DIR, 'dataset_statistics.pkl')

utils.save_dataset_statistics(data_provider, PICKLE_FILE_PATH, batch_size=1)

Let's load the dataset in the `ddsp` library and have a look at one of the examples.

In [ ]:
data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)

try:
  ex = next(iter(dataset))
except StopIteration:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

utils.specplot(ex['audio'])

f, ax = plt.subplots(3, 1, figsize=(14, 4))
x = np.linspace(0, 4.0, 1000)
ax[0].set_ylabel('loudness_db')
ax[0].plot(x, ex['loudness_db'])
ax[1].set_ylabel('F0_Hz')
ax[1].set_xlabel('seconds')
ax[1].plot(x, ex['f0_hz'])
ax[2].set_ylabel('F0_confidence')
ax[2].set_xlabel('seconds')
ax[2].plot(x, ex['f0_confidence'])


In [ ]:
ipd.Audio(ex['audio'], rate = sample_rate)

## Train Model

We will now train a "solo instrument" model. This means the model is conditioned only on the fundamental frequency (f0) and loudness with no instrument ID or latent timbre feature. If you uploaded audio of multiple instruemnts, the neural network you train will attempt to model all timbres, but will likely associate certain timbres with different f0 and loudness conditions. 

In [ ]:
%reload_ext tensorboard
import tensorboard as tb
tb.notebook.start('--logdir "{}"'.format(SAVE_DIR))

### We will now begin training. 

Note that we specify [gin configuration](https://github.com/google/gin-config) files for the both the model architecture ([solo_instrument.gin](TODO)) and the dataset ([tfrecord.gin](TODO)), which are both predefined in the library. You could also create your own. We then override some of the spefic params for `batch_size` (which is defined in in the model gin file) and the tfrecord path (which is defined in the dataset file). 

### Training Notes:
* Models typically perform well when the loss drops to the range of ~4.5-5.0.
* Depending on the dataset this can take anywhere from 5k-30k training steps usually.
* The default is set to 30k, but you can stop training at any time, and for timbre transfer, it's best to stop before the loss drops too far below ~5.0 to avoid overfitting.
* On the colab GPU, this can take from around 3-20 hours. 
* We **highly recommend** saving checkpoints directly to your drive account as colab will restart naturally after about 12 hours and you may lose all of your checkpoints.
* By default, checkpoints will be saved every 300 steps with a maximum of 10 checkpoints (at ~60MB/checkpoint this is ~600MB). Feel free to adjust these numbers depending on the frequency of saves you would like and space on your drive.
* If you're restarting a session and `DRIVE_DIR` points a directory that was previously used for training, training should resume at the last checkpoint.

In [ ]:
!ddsp_run \
  --mode=train \
  --alsologtostderr \
  --save_dir="$SAVE_DIR" \
  --gin_file=models/solo_instrument.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="TFRecordProvider.file_pattern='$TRAIN_TFRECORD_FILEPATTERN'"\
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=30000" \
  --gin_param="train_util.train.steps_per_save=300" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=10"

# TFRecordProvider: https://github.com/magenta/ddsp/blob/master/ddsp/training/data.py


## Resynthesis

Check how well the model reconstructs the training data

In [ ]:
data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_batch(batch_size=1, shuffle=False)

try:
  batch = next(iter(dataset))
except OutOfRangeError:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

# Parse the gin config.
gin_file = os.path.join(SAVE_DIR, 'operative_config-0.gin')
gin.parse_config_file(gin_file)

# Load model
model = ddsp.training.models.Autoencoder()
model.restore(SAVE_DIR)

# Resynthesize audio.
outputs = model(batch, training=False)
audio_gen = model.get_audio_from_outputs(outputs)
audio = batch['audio']

print('Original Audio')
specplot(audio)

print('Resynthesis')
specplot(audio_gen)

In [ ]:
print('Original Audio')
ipd.Audio(audio[0], rate = sample_rate)

In [ ]:
print('Resynthesis')
ipd.Audio(audio_gen[0], rate = sample_rate)

## Download Checkpoint

Below you can download the final checkpoint. You are now ready to use it in the [DDSP Timbre Tranfer Colab](https://colab.research.google.com/github/magenta/ddsp/blob/master/ddsp/colab/demos/timbre_transfer.ipynb).

In [ ]:
CHECKPOINT_ZIP = 'my_solo_instrument.zip'
latest_checkpoint_fname = os.path.basename(tf.train.latest_checkpoint(SAVE_DIR))
!cd "$SAVE_DIR" && zip $CHECKPOINT_ZIP $latest_checkpoint_fname* operative_config-0.gin dataset_statistics.pkl
!cp "$SAVE_DIR/$CHECKPOINT_ZIP" ./ #copy
# colab_utils.download(CHECKPOINT_ZIP)